<h2>PROGETTO MODULI E LIBRERIE PYTHON</h2>
<h3>Creazione di un <i>File organizer</i></h3>
<p>a cura di Paolo Acone</p>

**Step 1**

Script Python che itera in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposta nella relativa sottocartella. Se la sottocartella non esiste, lo script la crea automaticamente.

Durante il ciclo, lo script stampa le informazioni dei file: nome, tipo e dimensione in byte. 

Man mano che i file vengono  spostati, il documento *recap.csv* viene aggiornato con le stesse informazioni. 


In [ ]:
import os   #Il modulo os in Python fornisce un'interfaccia per interagire con il sistema operativo sottostante. In pratica, ti permette di eseguire operazioni sul tuo computer come se stessi usando il terminale o un'interfaccia grafica, ma direttamente dal tuo codice Python.
import shutil   #Il modulo shutil in Python offre una serie di funzioni di alto livello per eseguire operazioni su file e directory in modo più semplice e intuitivo rispetto al modulo os. In sostanza, shutil ti permette di copiare, spostare, rinominare e eliminare file e intere directory con poche righe di codice.
import csv  #Il modulo csv in Python fornisce strumenti per leggere e scrivere dati in formato CSV (Comma-Separated Values). All'interno di questo modulo, la classe csv.writer è specificamente progettata per scrivere dati in un file CSV.


folder_path= "files"
folder_list= sorted(os.listdir(folder_path))
recap_file = os.path.join(folder_path, "recap.csv")

# Verifica se il file recap.csv esiste già, altrimenti crea e scrive l'intestazione
if not os.path.exists(recap_file):
    with open(recap_file, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['file_name','type','size(B)'])  # Intestazione del CSV
        
# Definizione dei gruppi di file
file_groups = {
    'img': ['jpg', 'jpeg', 'png', 'gif', 'bmp'],
    'doc': ['pdf', 'docx', 'txt', 'xlsx', 'odt'],
    'audio': ['mp3', 'wav', 'flac']
}
# Lista per i file che non appartengono ad alcun gruppo
others = "others"

# Iterazione sui file nella cartella
for file in folder_list:
    file_path= os.path.join(folder_path, file)

    # Escludi il file recap.csv dal processo
    if file == "recap.csv":
        continue

    # Verifica se è un file
    if os.path.isfile(file_path):
        # Estrai l'estensione
        name, extension= os.path.splitext(file)
        extension=extension.lstrip('.').lower()
        file_size=os.path.getsize(file_path)

        # Trova il gruppo di estensioni a cui appartiene il file
        file_type = others
        for group, extensions in file_groups.items():
            if extension in extensions:
                file_type=group
                break

        # Creazione della sottocartella se non esiste
        subfolder_path= os.path.join(folder_path, file_type)
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Sposta il file nella sottocartella
        destination_path= os.path.join(subfolder_path, file)
        shutil.move(file_path, destination_path)

        # Aggiungi le informazioni al file recap.csv
        with open(recap_file, mode='a', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([file, file_type, file_size])

        # Stampa messaggio di conferma
        print(f'\nIl file:\n {file}, tipo: {file_type}, size: {file_size} B\n è stato spostato nella cartella: {file_type}')

**Step 2**

Creazione di un file eseguibile (*addfile.py*, situato nella stessa cartella del notebook) dotato di *interfaccia a linea di comando* (CLI)

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il file recap.csv

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare, comprensivo di formato (*nome_file*).  
Nel caso in cui il file passato come argomento non esiste, l'interfaccia lo comunica all'utente.


**Step 3**

Script che itera sulla sottocartella *img* e costruisce una tabella riassuntiva prodotta con la libreria *tabulate*.

Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore (per RGB) e opacità (per RGBA).

In [ ]:
from PIL import Image  # Libreria per la manipolazione di immagini
import numpy as np  # Libreria per operazioni numeriche su array
from tabulate import tabulate  # Libreria per creare tabelle formattate
import os  # Libreria per operazioni sui file e sui percorsi

# Percorso della directory contenente le immagini
path0= os.path.join('files','img')
lista_immagini= os.listdir(path0)

# Intestazioni per la tabella dei risultati
headers=['Nome file','Altezza (px)','Larghezza (px)','Grayscale, media','Rosso, media','Verde, media','Blu, media','Alpha, media']
data=[] # Lista che conterrà i dati delle immagini

# Iterazione su ciascun file nella directory
for imm in lista_immagini:
    img_path= os.path.join(path0, imm)      # Percorso completo dell'immagine
    img= Image.open(img_path)       # Apertura dell'immagine utilizzando PIL
    img_array= np.array(img)        # Conversione dell'immagine in un array NumPy per elaborazione
    
    if np.ndim(img_array)==3:   # Se l'immagine è a colori (3 dimensioni)
        media_rosso = np.mean(img_array[:, :, 0])  # Media del canale Rosso
        media_verde = np.mean(img_array[:, :, 1])  # Media del canale Verde
        media_blu = np.mean(img_array[:, :, 2])    # Media del canale Blu

        if img_array.shape[2]==4:      # Se l'immagine ha un canale alpha (RGBA)
            media_alpha = np.mean(img_array[:,:,3])     # Media del canale Alpha
            # Aggiungi i dati alla lista in formato RGBA
            array= [imm, img_array.shape[0], img_array.shape[1], None, media_rosso, media_verde, media_blu, media_alpha]  
        else:
            # Aggiungi i dati alla lista in formato RGB
            array= [imm, img_array.shape[0], img_array.shape[1],None,media_rosso, media_verde, media_blu]              
    elif np.ndim(img_array)==2:     # Se l'immagine è in scala di grigi (2 dimensioni)
        media_pixel= np.mean(img_array)     # Calcolo della media dei pixel
        # Aggiungi i dati alla lista in formato Grayscale
        array= [imm, img_array.shape[0], img_array.shape[1], media_pixel ]
    else:   # Se l'immagine non è riconosciuta
        print('Immagine non riconosciuta')
        continue    # Salta questa immagine

    # Aggiungi i dati raccolti alla lista principale
    data.append(array)

# Creazione della tabella utilizzando la libreria tabulate
table= tabulate(data, headers=headers, tablefmt="grid", floatfmt=".2f")
print(table)   